In [16]:
import pickle

import os 
import glob
import random
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import plyfile

import matplotlib.pyplot as plt
import matplotlib.patches as patches

import cv2
import numpy as np

import os.path as osp
import sys
ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import evaluation

data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
#scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
curr_scan_id = "02b33e01-be2b-2d54-93fb-4145a709cec5" 
new_scan_id =  "fcf66d8a-622d-291c-8429-0e1109c6bb26"
frame_number = "000008"
curr_frame_number = "000008"
new_frame_number = "000007"
patch_h= 9
patch_w = 16
patch_height = 60
patch_width = 60

/local/home/ekoller/BT


given the following: for the input image we have patchwise features. each patch has the same size. as the current image we have for each object in the scene the patch the following: a feature vector which is the average vector obtained by also dividing the image into the same size patches and then taking the vectors of the patches which belong to the objects based on the gt anno 2d
there is also a threshold involved

In [17]:
#cosine similarity between 2 vectors
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    return dot_product / (norm_vector1 * norm_vector2)


#iterate over each patch and for each patch give back the matched id. 
#assuming on a frame level not whole scan
def of_all_find_closes_pairs(obj_patches, patches):
    closest_object = np.zeros((patch_h,patch_w))
    closest_distance = np.zeros((patch_h,patch_w))
   
    #go over the image dict since we want to get the closest match of all the ids
    for h in range(patch_h):
        for w in range(patch_w):
            #initialize everything for this round
            min_distance = -1
            closest_id = None
            #access the patch feature
            patch_features = patches[h][w]

            #iterate over the object embeddings
            for obj_id in obj_patches:
                object_featrues = obj_patches[obj_id]

                #print("obj id vec shape", obj_vec.shape)
                #print("img id vec shape", img_vec.shape)
                cosine_similarity_all_patches = [cosine_similarity(object_featrues[i], patch_features[i]) for i in range(patch_features.shape[0])]
                average_cosine_similarity = np.mean(cosine_similarity_all_patches)
             
          
           
                #update
                if average_cosine_similarity> min_distance:
                    #print("update", average_cosine_similarity, "for patch ", h, "_", w)
                    min_distance = average_cosine_similarity
                    closest_id= obj_id

            #set the values for the image
            print(" final object id", closest_id, "for patch ", h, "_", w)
            closest_object[h][w] = closest_id
            closest_distance[h][w] = min_distance
 

        #closest_pairs[img_id] = (closest_id, min_distance)

    return closest_object, closest_distance
    


In [18]:
#for a given scene get the colours of the differnt object_ids
def get_id_colours(data_dir,scan_id):
    #access the mesh file to get the colour of the ids
    mesh_file = osp.join(data_dir,"scenes", scan_id, "labels.instances.annotated.v2.ply")
    ply_data = plyfile.PlyData.read(mesh_file)
    # Extract vertex data
    vertices = ply_data['vertex']
    vertex_count = len(vertices)
    
    # Initialize dictionary to store object_id -> color mappings
    object_colors = {}
    
   # Iterate through vertices
    for i in range(vertex_count):
        vertex = vertices[i]
        object_id = vertex['objectId']
        color = (vertex['red'], vertex['green'], vertex['blue'])
        
        # Check if object_id already in dictionary, otherwise initialize a Counter
        if object_id in object_colors:
            object_colors[object_id][color] += 1
        else:
            object_colors[object_id] = Counter({color: 1})
    
    # Convert Counter to dictionary with most frequent color
    for object_id, color_counter in object_colors.items():
        most_common_color = color_counter.most_common(1)[0][0]
        object_colors[object_id] = np.array(most_common_color[::-1])
    
    return object_colors

colors = get_id_colours(data_dir,curr_scan_id)
print("cur scan id colors", len(colors.keys()))
colors = get_id_colours(data_dir,new_scan_id)
print("new scan id colors", len(colors.keys()))


cur scan id colors 28
new scan id colors 15


In [19]:
#this function creates a color image of the size 960x540 from the patches
def create_color_img_from_obj_id(data_dir,scan_id,obj_id_mat):
    #access the mesh file to get the colour of the ids
    colour_dict = get_id_colours(data_dir, scan_id)
    #initialize the new matrix 
    og_height, og_width = obj_id_mat.shape
    new_height = og_height * patch_height
    new_width = og_width * patch_width

    colour_mat = np.zeros((new_height,new_width,3))

    #go over each element and assign the colour of the dictionary
    for h in range(og_height):
        for w in range(og_width):
            id = obj_id_mat[h][w]
            colour = colour_dict[id]
            colour_mat[h*patch_height:(h+1)*patch_height, w*patch_width:(w+1)*patch_width] = colour


    return colour_mat


    

In [20]:
#given a gt patchwise matrix and a newly calculated matrix compute the accuracy
def get_accuracy(gt_patches, computed_patches):
    #make sure we dont do something dumm lol
    assert gt_patches.shape == computed_patches.shape, "Matrices must have the same shape"

    # Flatten matrices to iterate over each element
    flat_gt = gt_patches.flatten()
    flat_comp = computed_patches.flatten()
    
    total_instances = len(flat_gt)
    correct_instances = 0
    
    # Count correct instances where IDs match
    for id_gt, id_comp in zip(flat_gt, flat_comp):
        if id_gt == id_comp:
            correct_instances += 1
    
    # Compute accuracy
    accuracy = correct_instances / total_instances
    
    return accuracy

In [22]:

#given a gt patchwise matrix and a newly calculated matrix compute the accuracy
def get_accuracy(gt_patches, computed_patches):
    #make sure we dont do something dumm lol
    assert gt_patches.shape == computed_patches.shape, "Matrices must have the same shape"

    # Flatten matrices to iterate over each element
    flat_gt = gt_patches.flatten()
    flat_comp = computed_patches.flatten()
    
    total_instances = len(flat_gt)
    correct_instances = 0
    
    # Count correct instances where IDs match
    for id_gt, id_comp in zip(flat_gt, flat_comp):
        if id_gt == id_comp:
            correct_instances += 1
    
    # Compute accuracy
    accuracy = correct_instances / total_instances
    
    return accuracy

new_scan_id = curr_scan_id
new_frame_number = frame_number

"""
initialize here current = new scan id since we are looking at the gt
"""
#read in all the data needed for the evaluation currently this tests the sam input against the ground truth of the input

#get the dinofeatures for each patch
input_featues_path = osp.join(data_dir,"files/Features2D/patch/DinoV2", new_scan_id + ".pkl")
with open(input_featues_path, 'rb') as file:
    input_features = pickle.load(file)


#get the dino features for each object
curr_featues_path = osp.join(data_dir,"files/Features2D/segmented_patch/DinoV2", curr_scan_id + ".pkl")
with open(curr_featues_path, 'rb') as file:
    curr_features = pickle.load(file)

"""
compute the best match for each input on a patchwise level
"""
#for each patch compute the corresponding object id, return matrix on patchwise level
print("size of curent features of object", curr_features[frame_number][1].shape)
print("size of input patch features", len(input_features[new_frame_number]),len(input_features[new_frame_number][0]))
id_matches_obj, match_dist = of_all_find_closes_pairs(curr_features[frame_number], input_features[new_frame_number])

#print("match dist", match_dist)
#print("id mathces", id_matches_obj)

new_img_colour = create_color_img_from_obj_id(data_dir,new_scan_id, id_matches_obj)

#get the gt input patches and also turn them into bigger images
gt_input_patchwise_path =  osp.join(data_dir,"files/patch_anno/patch_anno_16_9", new_scan_id + '.pkl')
with open(gt_input_patchwise_path, 'rb') as file:
    gt_input_patchwise = pickle.load(file)
gt_input_colour = create_color_img_from_obj_id(data_dir,new_scan_id,gt_input_patchwise[frame_number])


accuracy = get_accuracy(gt_input_patchwise[frame_number],id_matches_obj)
print("the accuracy for the current features of the sam boundingboxes vs the gt boundingboxes is", accuracy)


"""
display the newly computed images next to each other
"""

#display the gt_image and the new patchwise image next to each other
title1 = "gt_patces"
title2 = "predicted_patches"

# Create a blank canvas to combine images horizontally
height = max(gt_input_colour.shape[0], new_img_colour.shape[0])  # Max height of both images
width = gt_input_colour.shape[1] + new_img_colour.shape[1] + 20  # Total width of both images with a small gap
combined_image = np.zeros((height, width, 3), dtype=np.uint8)

# Place images with titles on the blank canvas
combined_image[:gt_input_colour.shape[0], :gt_input_colour.shape[1]] = gt_input_colour
combined_image[:new_img_colour.shape[0], gt_input_colour.shape[1] + 20:] = new_img_colour

# Add titles to the images
cv2.putText(combined_image, title1, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(combined_image, title2, (gt_input_colour.shape[1] + 30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

# Display the combined image
cv2.imshow('Two Images Side by Side', combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()



size of curent features of object (81, 1536)
size of input patch features 18 32
 final object id 4 for patch  0 _ 0
 final object id 4 for patch  0 _ 1
 final object id 4 for patch  0 _ 2
 final object id 4 for patch  0 _ 3
 final object id 4 for patch  0 _ 4
 final object id 4 for patch  0 _ 5
 final object id 4 for patch  0 _ 6
 final object id 4 for patch  0 _ 7
 final object id 4 for patch  0 _ 8
 final object id 4 for patch  0 _ 9
 final object id 15 for patch  0 _ 10
 final object id 15 for patch  0 _ 11
 final object id 15 for patch  0 _ 12
 final object id 4 for patch  0 _ 13
 final object id 15 for patch  0 _ 14
 final object id 15 for patch  0 _ 15
 final object id 4 for patch  1 _ 0
 final object id 4 for patch  1 _ 1
 final object id 15 for patch  1 _ 2
 final object id 4 for patch  1 _ 3
 final object id 4 for patch  1 _ 4
 final object id 4 for patch  1 _ 5
 final object id 4 for patch  1 _ 6
 final object id 4 for patch  1 _ 7
 final object id 4 for patch  1 _ 8
 final o